In [1]:
import pandas as pd
from conllu import parse
from nltk import DependencyGraph
from ufal.udpipe import Model, Pipeline
import re
from transformers import AutoTokenizer
import numpy as np

In [2]:
df = pd.read_excel('13112022_main_full_fix.xlsx')
df

,Unnamed: 0,q,a,tq,ta
0,0,"Здравствуйте, подскажите пожалуйста, я обучала...",Здравствуйте. По данному вопросу обратитесь по...,"[38209., 5750., 343., 64617., 38645., 56033., ...","[70433., 33013.]"
1,1,"Здраствуйте, если ты уже зачислен на первый ку...",Здравствуйте. Вам необходимо сообщить об этом ...,"[45783., 33649., 733., 33980., 2993.]","[18016., 29619., 38602.]"
2,2,Здравствуйте! В магистратуре можно ли поменять...,Здравствуйте. По данному вопросу обратитесь по...,"[5081., 42173., 15234., 14159.]","[70433., 33013.]"
3,3,"Добрый день, а можно узнать стипендию на факул...",Здравствуйте. По данному вопросу вам необходим...,"[5081., 4045., 30799., 68380.]","[18016., 29619., 32373.]"
4,4,"Здравствуйте, как получить справку для родител...",Здравствуйте. По данному вопросу вам необходим...,"[18348., 53039., 31590., 48341., 1046.]","[18016., 29619., 32373.]"
...,...,...,...,...,...
44894,44894,"Скажите пожалуйста,какие нужно в этом году сда...","Надо сдавать экзамены по русскому языку, матем...","[21211., 33013., 872., 8656., 41999., 45108.]","[29624., 41999., 45108.]"
44895,44895,какие мне экзамены нужно сдавать по егэ на спе...,Надо сдавать в форме и по материалам ЕГЭ матем...,"[13267., 21211., 21211., 45108., 20284., 41999...","[29624., 72981., 12654., 3413., 22850., 27207...."
44896,44896,"Если да, то когда?",В город Семей Республики Казахстан выезд прием...,['!'],"[31408., 33997.]"
44897,44897,Поступать нужно по сертификату о сдаче ЕГЭ или...,Свидетельство ЕГЭ с результатами экзаменов по ...,"[38600., 21211., 41999., 45108.]","[52925, 25992, 45968, 49216]"


In [3]:
import difflib

def mySort(s1, s2):
  matcher = difflib.SequenceMatcher(None, s1, s2)
  return matcher.ratio()

In [4]:
def get_sov(sent):
    graph = DependencyGraph(tree_str=sent)
    sov = {}
    for triple in graph.triples():
        if triple:
            if triple[0][1] == 'VERB':
                sov[triple[0][0]] = {'subj':'','obj':''}
    for triple in graph.triples():
        if triple:
            if triple[1] == 'nsubj':
                if triple[0][1] == 'VERB':
                    sov[triple[0][0]]['subj']  = triple[2][0]
            if 'obj' in triple[1]:
                if triple[0][1] == 'VERB':
                    sov[triple[0][0]]['obj'] = triple[2][0]
    return sov

In [5]:
model = Model.load('russian-ud-2.0-170801.udpipe')
tokenizer = AutoTokenizer.from_pretrained("cointegrated/rubert-tiny2")
def tokenText(text):
    arrParsed = []
    longParsed = ''

    sent = str(text)
    sent = re.sub("\s\s+", ' ', sent)
    sent = sent.lower()
    sent = sent.strip()
    sent = sent.replace(u'\xa0', u' ')
    sentS = re.split(";|! |\?|\. ", sent)
    sentS = list(filter(None, sentS))

    for nSentS in sentS:
        nSentS = re.sub(r'[^\w\s]','', nSentS) 

        pipeline = Pipeline(model, 'tokenize', Pipeline.DEFAULT, Pipeline.DEFAULT, Pipeline.DEFAULT)
        parsed = pipeline.process(nSentS)

        # костыли для dependency graph
        parsed = '\n'.join([line for line in parsed.split('\n') if not line.startswith('#')])
        parsed = parsed.replace('\troot\t', '\tROOT\t')

        if (len(longParsed) < len(parsed)):
            longParsed = parsed

        arrParsed.append(parsed)
    arrParsed = list(filter(None, arrParsed))

    arrSov = []
    for nArrParsed in arrParsed:
        sov = get_sov(nArrParsed)
        arrSov.append(sov)

    bigSov = arrSov[0]
    if (len(arrSov) > 1):
        for i in range(1, len(arrSov)):
            bigSov = { **bigSov, ** arrSov[i] }

    tokenizedA = []
    if (bigSov == {}):
        arrGraph = []
        graph = DependencyGraph(tree_str=longParsed)
        for i in range(0, len(list(graph.triples()))):
            for j in range(0, len(list(graph.triples())[0])):
                if (list(graph.triples())[i][j][1] == 'NOUN'):
                    arrGraph.append(list(graph.triples())[i][j][0])
        arrGraph = set(arrGraph)
        if (len(arrGraph) == 0):
            tokenizedA.append('!');
        else:
            for nArrGraph in arrGraph:
                tokenizedA.append(tokenizer.encode(nArrGraph, add_special_tokens=False))
            tokenizedA = np.concatenate(tokenizedA, axis=0, out=None, dtype=None, casting="same_kind")
    else:
        for k, v in bigSov.items(): 
            tokenizedA.append(tokenizer.encode(k, add_special_tokens=False))
            tokenizedA.append(tokenizer.encode(v['subj'], add_special_tokens=False))
            tokenizedA.append(tokenizer.encode(v['obj'], add_special_tokens=False))
        tokenizedA = np.concatenate(tokenizedA, axis=0, out=None, dtype=None, casting="same_kind")

    return tokenizedA

In [12]:
text = 'До какого числа можно принести аттестат?'
tToken = tokenText(text)

tq = []
index = 0
probability = 0
textQ = ''
textA = ''
for i in range(0, len(df)):
    if (probability <= mySort(tToken, eval(df['tq'][i]))):
        tq = eval(df['tq'][i])
        index = i
        textQ = df['q'][i]
        textA = df['a'][i]
        probability = mySort(tToken, eval(df['tq'][i]))
print(text, '- Текст основного вопроса')
print(tToken, '- Токенизированный текст основного вопроса')
print(probability*100, '% - Процент сходства с найденным вопросом')
print(index, '- Номер найденного вопроса')
print(textQ, '- Текст найденного вопроса')
print(tq, '- Токенизированный найденный вопрос')
print(textA, '- Ответ к найденному вопросу')

До какого числа можно принести аттестат? - Текст основного вопроса
[ 5081. 39974. 63570.] - Токенизированный текст основного вопроса
66.66666666666666 % - Процент сходства с найденным вопросом
25821 - Номер найденного вопроса
Добрый день, оригиналы документов можно принести 4 августа. А в какое время? когда крайний срок? - Текст найденного вопроса
[5081.0, 25982.0, 39974.0] - Токенизированный найденный вопрос
Подлинники документов можно будет предоставить в приёмную комиссию 4 августа с 9-00 до 15-00. - Ответ к найденному вопросу
